In [1]:
import itertools
import numpy as np
import os
import random
import sys
import tensorflow as tf
import plotting
import matplotlib.pyplot as plt
import vcamp
import pandas as pd


/usr/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
batch_size = 3
action_size = 4
action_samples = 2

s = np.array([[1, 1, 11 ,10, 1],[2, 2, 22 ,20, 2],[3, 3, 33 ,30, 3]])
a = np.array([[1, 1, 1, 1],[1 ,10, 1, 1],[2, 2, 2,1],[2 ,20, 2,1],[3, 3, 3,1],[3 ,30, 3,1]])
v = np.array([1, 2, 10, 2 ,1, 2])

s = np.repeat(s,action_samples, axis=0)
a = a.reshape((batch_size,action_samples,action_size))
v = v.reshape((batch_size,action_samples,1))

max_loc = np.argmax(v, axis=1)
max_loc = max_loc.reshape(batch_size)

for i in range(batch_size):
    max_loc[i] = max_loc[i] + i*action_samples

a = a.reshape((batch_size*action_samples,action_size))
print (max_loc)
print (a)
print (max_loc.shape)


a = a[max_loc]
print(a)
print (a.shape)



[1 2 5]
[[ 1  1  1  1]
 [ 1 10  1  1]
 [ 2  2  2  1]
 [ 2 20  2  1]
 [ 3  3  3  1]
 [ 3 30  3  1]]
(3,)
[[ 1 10  1  1]
 [ 2  2  2  1]
 [ 3 30  3  1]]
(3, 4)


In [3]:
# create env and load POF 
design_table = pd.read_csv("/home/aida/workspace/DeepLearning/ssvcamp-ngspice/design_table.dat", sep=' ')
env = vcamp.VcAmpRLEnv()

# sort columns and convert to numpy
solutions = design_table[list(env.parameters)].values

In [9]:
transition_table = None

for r in range(solutions.shape[0]):
    state = env.reset(values=solutions[r,:])
    gsum_state = env.current_performance
    actions = env.sample_action(128)
    for a in actions:
        next_state, _,_,_ = env.step(a)
        next_state_gsum = env.current_performance
        #forward transition
        tran_f = vcamp.transition_as_array(state, gsum_state, next_state, next_state_gsum, a)
        #backwards transition
        tran_b = vcamp.transition_as_array(next_state, next_state_gsum, state, gsum_state, a*(-1.0))
        
        transition_table = vcamp.update_tran_table(transition_table, tran_f)
        transition_table = vcamp.update_tran_table(transition_table, tran_b)

        #update
        gsum_state = next_state_gsum
        state      = next_state
        
        
    


In [10]:
df= pd.DataFrame(data=transition_table, columns= vcamp.transition_header(state,a))
print(df)


      gsum_state  gsum_next_state  action_0  action_1  action_2  action_3  \
0       0.000000         0.000000 -0.197381  0.274359 -0.132770  0.035527   
1       0.000000         0.000000  0.197381 -0.274359  0.132770 -0.035527   
2       0.000000        -0.047769 -0.117744  0.172663  0.499508  0.120782   
3      -0.047769         0.000000  0.117744 -0.172663 -0.499508 -0.120782   
4      -0.047769         0.000000 -0.176408 -0.083206 -0.152944 -0.224476   
5       0.000000        -0.047769  0.176408  0.083206  0.152944  0.224476   
6       0.000000        -0.337408  0.155099 -0.436054 -0.294942  0.415585   
7      -0.337408         0.000000 -0.155099  0.436054  0.294942 -0.415585   
8      -0.337408        -0.463369  0.418105  0.128481 -0.033690 -0.144798   
9      -0.463369        -0.337408 -0.418105 -0.128481  0.033690  0.144798   
10     -0.463369        -0.742027  0.376494  0.273097  0.168508  0.353708   
11     -0.742027        -0.463369 -0.376494 -0.273097 -0.168508 -0.353708   

In [11]:
df.to_csv("transition_table.csv")